In [1]:
import pdfplumber
import groq
import ollama
import os
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_ollama import ChatOllama
from langchain_core.prompts.prompt import PromptTemplate

In [89]:
pdf=pdfplumber.open("test3.pdf")
pages=pdf.pages

In [90]:
text=""
for i in range(6):
    text+=pages[i].extract_text()


In [9]:
from groq import Groq

In [10]:
client=Groq(api_key="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")

In [102]:
prompt='''
you are a medical report parser.
Give me the output in json format strictly with no preamble.
If you dont get any value assign it None
Extract the following fields
    name of the doctor with key as "doctor_name"
    date of report with key as "date_of_report" with dd/mm/yy format
    date of collection with key as "date_of_collections" with dd/mm/yy format
    hemoglobin with key as "hemoglobin"
    rbc count with key as "rbc_count"
    wbc count with key as "wbc_count"
    pcv with key as "pcv"
    iron with key as "iron"
    sodium with key "sodium"
    pottasium with key "pottasium"
    phosphorus with key "phosphorus"
    chloride with key "chloride"
    platelet count with key as "platelet_count"
    bilirubin total with key as "bilirubin_total"
    bilirubin direct with key as "bilirubin_direct"
    bilirubin indirect with key as "bilirubin_indirect"
    protiens with key as "protiens"
    calcium with key as "calcium"
    albumin with key as "albumin"
    Globulin with key as "globulin"
    Blood Urea with the key "blood_urea"
    Blood Urea Nitrogen with the key "blood_urea_nitrogen"
    S. Creatinine with the key "s_creatinine"
    S. Uric Acid with the key "s_uric_acid"
    S. Phosphorus with the key "s_phosphorus"
    Neutrophils with the key "neutrophils".
    Lymphocytes with the key "lymphocytes".
    Sr. Cholesterol with the key "sr_cholesterol".
    HDL Cholesterol with the key "hdl_cholesterol".
    fasting sugar with key "fasting_sugar"
    after lunch sugar with key "after_lunch_sugar"
    summary of the entire report of each section in short with key "summary"
Strictly give me in proper json format  and it should not be nested json with no preamble.
''' 

In [103]:
chat_completion=client.chat.completions.create(
    messages=[{'role':"system","content":prompt},{"role":"user","content":text}],
    model="llama3-70b-8192"
)

In [105]:
data=eval(chat_completion.choices[0].message.content)

SyntaxError: invalid syntax (<string>, line 1)

In [81]:
data

{'doctor_name': 'Raihan Ahmed Siddiqui',
 'date_of_report': '03-Aug-2024',
 'date_of_collections': '02-Aug-2024',
 'hemoglobin': 12.1,
 'rbc_count': 4.72,
 'wbc_count': 9700,
 'pcv': 38,
 'platelet_count': 219000,
 'bilirubin_total': 0.57,
 'bilirubin_direct': 0.19,
 'bilirubin_indirect': 0.38,
 'protiens': 6.18,
 'calcium': 8.19,
 'albumin': 3.16,
 'globulin': 3.02,
 'blood_urea': 21,
 'blood_urea_nitrogen': 9.81,
 's_creatinine': 0.98,
 's_uric_acid': 4.25,
 's_phosphorus': 3.84,
 'neutrophils': 91,
 'lymphocytes': 6,
 'sr_cholesterol': None,
 'hdl_cholesterol': None,
 'fasting_sugar': None,
 'after_lunch_sugar': None,
 'summary': "The patient's Hemoglobin, RBC Count, and Platelet Count are slightly low. The patient also has Hypochromia and Mild Microcytosis in RBC Morphology. The patient's Bilirubin Total, Bilirubin Direct, and Bilirubin Indirect are within normal range. The patient's Blood Urea and Blood Urea Nitrogen are slightly high. The patient's S. Creatinine, S. Uric Acid, an

In [95]:
def process_pdf(text):
    try:
        chat_completion=client.chat.completions.create(
            messages=[{'role':"system","content":prompt},{"role":"user","content":text}],
            model="llama3-70b-8192"
            )
        data=eval(chat_completion.choices[0].message.content)
        return {"data":data,"status":True}
    except:
        return {"status":False,"data":chat_completion.choices[0].message.content}

In [106]:
process_pdf(text)

{'data': {'doctor_name': 'CLIENT',
  'date_of_report': '08/12/2019',
  'date_of_collections': '08/12/2019',
  'hemoglobin': 15.2,
  'rbc_count': 5.46,
  'wbc_count': 6.1,
  'pcv': 46.8,
  'iron': 82,
  'sodium': 139,
  'pottasium': 4.5,
  'phosphorus': 341,
  'chloride': 99,
  'platelet_count': 228,
  'bilirubin_total': 0.4,
  'bilirubin_direct': 0.1,
  'bilirubin_indirect': 0.3,
  'protiens': 8.6,
  'calcium': 9.4,
  'albumin': 4.4,
  'globulin': 4.2,
  'blood_urea': 8.0,
  'blood_urea_nitrogen': 8.0,
  's_creatinine': 0.9,
  's_uric_acid': 6.8,
  's_phosphorus': 341,
  'neutrophils': 56,
  'lymphocytes': 32,
  'sr_cholesterol': 228,
  'hdl_cholesterol': 50,
  'fasting_sugar': 84,
  'after_lunch_sugar': 125.5,
  'summary': 'The report shows some abnormal values, including Hemoglobin, Mean Corpuscular Volume, Mean Corpuscular Hemoglobin, Platelet Count, Total Protein, Globulin, Fasting Sugar, Blood Urea Nitrogen, Creatinine, Uric Acid, Thyroid Stimulating Hormone, Cholesterol, Triglyce

In [25]:
type_prompt='''
you are a medical report classifier.
you will be given a medical report in text and you have to classify it in blood_report or other.
if there are values of hemoglobin,rbc count wbc count,platelet count bilirubin etc then it is a blood report else other.
Give me the result in Json format with key as "type" with no preamble and no nested json.
'''

In [28]:
pdf=pdfplumber.open("test3.pdf")
pages=pdf.pages

In [29]:
text=""

In [30]:
for page in pages[:2]:
    text+=page.extract_text()

In [31]:
text

"DIAGNOSTIC REPORT\nCLIENT CODE : C000096169\nCert. No. MC-2015\nCLIENT'S NAME AND ADDRESS :\nNUTRIWELL HEALTH INDIA PVT LTD SRL LIMITED\n227 OKHLA PHASE lll, FIRST FLOOR, SOUTH DELHI, SRL,REFERENCE LAB, GP-26, MARUTI INDUSTRIAL ESTATE,UDYOG\nVIHAR,SECTOR-18,\nDELHI 110020 GURGAON, 122015\nDELHI INDIA HARYANA, INDIA\n11-41343500 9873863338 Tel : 1800-222-000, 1800-102-8282, Fax : 0124-4591001 CIN -\nU74899PB1995PLC045956\nEmail : connect@srl.in\nPATIENT NAME : DEVENDER SINGH PATIENT ID :\nACCESSION NO : 0009SL015385 AGE : 51 Years SEX : Male DATE OF BIRTH :\nDRAWN : RECEIVED : 08/12/2019 15:48 REPORTED : 08/12/2019 19:45\nREFERRING DOCTOR : CLIENT PATIENT ID :\nTest Report Status Final Results Biological Reference Interval Units\nCOMPLETE CARE TOTAL\nBLOOD COUNTS\nHEMOGLOBIN 15.2 13.0 - 17.0 g/dL\nMETHOD : PHOTOMETRIC MEASUREMENT\nREDBLOODCELLCOUNT 5.46 4.5 - 5.5 mil/µL\nMETHOD : COULTER IMPEDENCE PRINCIPLE\nWHITEBLOODCELLCOUNT 6.10 4.0 - 10.0 thou/µL\nMETHOD : COULTER IMPEDENCE PRINCI

In [32]:
chat_completion=client.chat.completions.create(
    messages=[{'role':"system","content":type_prompt},{"role":"user","content":text}],
    model="llama3-70b-8192"
)

In [33]:
eval(chat_completion.choices[0].message.content)

{'type': 'blood_report'}

In [34]:
other_prompt='''
you are a medical report summarizer.
summarize the report and give the summary in json format with key as "summary".
Only give me json with no preamble
'''